### AST 4: ETL concepts and pipeline

## Learning Objectives

At the end of the experiment, you will be able to:

* use Spark’s built-in and external data sources to read, refine, and write data in different file formats as part of the extract, transform, and load (ETL) tasks
* perform complex data exploration and analysis using Spark SQL

### Introduction

ETL (Extract, Transform, and Load) is the procedure of migrating data from one system to another.
* Data **extraction** is the process of retrieving data out of homogeneous or heterogeneous sources for further data processing and data storage.
* During data **transformation**, the data is cleaned and incorrect or inaccurate records are modified or deleted.
* Finally, the processed data is **loaded** (or stored) into a target system such as a data warehouse or NoSQL database or RDBMS.

Data engineers use Spark because it provides a simple way to parallelize computations and hides all the complexity of distribution and fault tolerance. This leaves them free to focus on using high-level DataFrame-based APIs and domain-specific language queries to do ETL, reading and combining data from multiple sources.

Here we will consider tabular data to do ETL operations. Starting from data extraction, we will perform various transformations and try to gain some insights from it and then load it to a NoSQL database or store it in different file formats.

### Dataset

The dataset chosen for this assignment is [Productivity Prediction of Garment Employees](https://archive.ics.uci.edu/ml/datasets/Productivity+Prediction+of+Garment+Employees). The dataset is made up of 1197 records and 15 columns. It includes important attributes of the garment manufacturing process and the productivity of the employees. The dataset contains records of three months (Jan to Mar 2015) with 2 distinct departments, 12 unique team numbers and 5 different quarters. Some of the features are listed below:

* date: Date in MM-DD-YYYY
* day: Day of the Week
* quarter: A portion of the month. A month was divided into four quarters
* department: Associated department with the instance
* team: Associated team number with the instance
* no_of_workers: Number of workers in each team
* no_of_style_change: Number of changes in the style of a particular product
* targeted_productivity: Targeted productivity set by the Authority for each team for each day.
* smv: Standard Minute Value, it is the allocated time for a task
* wip: Work in progress. Includes the number of unfinished items for products
* overtime: Represents the amount of overtime by each team in minutes
* incentive: Represents the amount of financial incentive (in BDT) that enables or motivates a particular course of action.
* idletime: The amount of time when the production was interrupted due to several reasons
* idlemen: The number of workers who were idle due to production interruption
* actual_productivity: The actual % of productivity that was delivered by the workers. It ranges from 0-1.

Date, quarter, department, and day are object datatypes and the rest are int or float types.

To know more about the dataset click [here](https://archive.ics.uci.edu/ml/datasets/Productivity+Prediction+of+Garment+Employees).

### Setup Steps:

### Install Pyspark

In [ ]:
!pip install pyspark

### Import required packages

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd

### Start a Spark Session

Spark session is a combined entry point of a Spark application, which came into implementation from Spark 2.0. Instead of having various context, everything is now encapsulated in a **Spark session**.

In [ ]:
# Start spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ETL').getOrCreate()
spark

### Tabular Data Analytics

#### Extract data into PySpark
To load the dataset we will use the read.csv module.  The inferSchema parameter provided will enable Spark to automatically determine the data type for each column.

In [ ]:
df = spark.read.csv('garments_worker_productivity.csv', header=True, inferSchema= True)

#### Transforming Data

* Display first few rows of the data

In [ ]:
df.show(5)

In the above output, the wip (work in progress) column contains null values. We need to check for other columns as well.
* Display total number of rows

In [ ]:
df.count()

* Check for Null values in each column

In [ ]:
df.select([(count(when(isnan(c) | col(c).isNull(), c))/1197).alias(c) for c in df.columns]).show()

The wip column contains around 42% null values so we can drop that column.
* Drop the wip column having Null values

In [ ]:
df1 = df.drop('wip')

In [ ]:
# Recheck for null values
df1.select([(count(when(isnan(c) | col(c).isNull(), c))/1197).alias(c) for c in df1.columns]).show()

Let's see the distinct values in department column.
* Display distinct `department` from dataframe

In [ ]:
# Display count of distinct 'department'
df1.select('department').distinct().count()

In [ ]:
# Display distinct values for 'department' column
df1.select('department').distinct().show()

There is ambiguity in distinct department values and it needs to be taken care of.
* Transform department column

In [ ]:
# Removing trailing spaces from both sides using department column
df2 = df1.withColumn('department', trim(col('department')))

In [ ]:
# Display distinct values for 'department' column
df2.select('department').distinct().show()

In [ ]:
# Replacing department value from 'sweing' to 'sewing'
df3 = df2.withColumn('department', regexp_replace(col('department'), 'sweing', 'sewing'))

In [ ]:
# Display distinct values for 'department' column
df3.select('department').distinct().show()

* Check for duplicate values in data

In [ ]:
cols = df3.columns
if df3.count() > df3.dropDuplicates(cols).count():
    print('Data has duplicates')
else:
  print('Data has no duplicates')

If data has duplicate values, run the below cell by uncommenting it

In [ ]:
# df3 = df3.dropDuplicates(df3.columns)

After removing duplicates, let's take a look at the datatypes of our columns.
* Display data types of dataframe columns

In [ ]:
# Print the data types
df3.dtypes

Here, the 'date' column has a string datatype. In order to perform analytics involving year and month, we need to convert it into timestamp datatype.
* Transform the `date` column from string type to Spark `timestamp` data type

In [ ]:
df4 = df3.withColumn("date", to_timestamp(col("date"), "M/d/yyyy"))

In [ ]:
df4.dtypes

Now we can use the `year()` SQL Spark function on the Timestamp column data type `date`.
* Display how many distinct years of data is in the dataset

In [ ]:
df4.select(year('date')).distinct().orderBy(year('date')).show()

We see that all the records are from the year 2015.

Similar to year() we can use the `month()` SQL Spark function on the Timestamp datatype column `date`.
* Display how many distinct months of data is in the dataset

In [ ]:
df4.select(month('date')).distinct().orderBy(month('date')).show()

We see that the data was collected in the months January to March.

* Check in which month the productivity was maximum

In [ ]:
df_d = df4.groupby(month('date')).avg().select(['month(date)', 'avg(actual_productivity)'])
df_d.show()
sns.barplot(x = df_d.toPandas()['month(date)'], y= df_d.toPandas()['avg(actual_productivity)'])

From the above plot, it can be seen that in January the productivity was little higher than remaining two.

Let's gain few more insights from the data
* Display the incentives paid to different teams

In [ ]:
df_i = df4.groupby('team').avg().select(['team', 'avg(incentive)'])
df_i.show()
sns.barplot(x = df_i.toPandas()['team'], y= df_i.toPandas()['avg(incentive)'])

So on average team 9 received the highest incentive.
* Display number of workers in each Team

In [ ]:
df_w = df4.groupby('team').sum().select(['team', 'sum(no_of_workers)'])
df_w.show()
sns.barplot(x = df_w.toPandas()['team'], y= df_w.toPandas()['sum(no_of_workers)'])

From the above plot, it can be seen that team 6 and 12 have less number of workers compared to other teams.

Let's find out how much it costs for the first quarter of the first month.
* Display the `incentive` paid for the first quarter of the first month

In [ ]:
df_q = df4.select(month("date"), "quarter", "incentive").where((col('month(date)') == 1) & (col("quarter") == "Quarter1"))
df_q.show(5)
df_q.groupby('quarter').sum().select('sum(incentive)').show()

Before fitting to a model, the outlier removal and feature scaling of data are important.
* Check for outliers

In [ ]:
df4.toPandas().boxplot()
plt.xticks(rotation= 90)
plt.show()

* Handling outliers

Here instead of removing the outliers we will change their values to upper bound and lower bound depending on whether the value is higher than upper bound or lower than lower bound respectively.

In [ ]:
df5 = df4
outlier_cols = ["targeted_productivity", "smv", "over_time", "incentive", "idle_time", "idle_men", "no_of_style_change", "actual_productivity"] # Columns with outliers
def handle_outliers(df, colm):
    df = df.toPandas()
    q1 = df.describe()[colm].loc["25%"]
    q3 = df.describe()[colm].loc["75%"]
    iqr = q3 - q1
    lower_bound = q1 - (1.5 * iqr)
    upper_bound = q3 + (1.5 * iqr)
    for i in range(len(df)):
      if df.loc[i,colm] > upper_bound:
        df.loc[i,colm]= upper_bound
      if df.loc[i,colm] < lower_bound:
        df.loc[i,colm]= lower_bound
    return spark.createDataFrame(df)

for colm in outlier_cols:
    df5 = handle_outliers(df5, colm)

In [ ]:
# Recheck for outliers
df5.toPandas().boxplot()
plt.xticks(rotation= 90)
plt.show()

* Scaling features

In [ ]:
# Display the statistics of dataframe
df5.toPandas().describe()

In [ ]:
# Specify columns that need to be scaled
columns = ["smv", "over_time", "incentive", "no_of_workers"]

In [ ]:
print("Before Scaling :")
df5.show(5)
df6 = df5

from pyspark.ml.feature import MinMaxScaler, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import udf, round
from pyspark.sql.types import DoubleType

# UDF for converting column type from vector to double type
unlist = udf(lambda x: float(list(x)[0]), DoubleType())
spark.udf.register("unlist", unlist)

# Iterating over columns to be scaled
for i in columns:
    # VectorAssembler Transformation - Converting column to vector type
    assembler = VectorAssembler(inputCols=[i],outputCol=i+"_Vect")

    # MinMaxScaler Transformation
    scaler = MinMaxScaler(inputCol=i+"_Vect", outputCol=i+"_Scaled")

    # Pipeline of VectorAssembler and MinMaxScaler
    pipeline = Pipeline(stages=[assembler, scaler])

    # Fitting pipeline on dataframe
    df6 = pipeline.fit(df6).transform(df6).withColumn(i+"_Scaled", unlist(i+"_Scaled")).drop(i+"_Vect")

print("After Scaling :")
df6.show(5)

Let's create a new `id` column that will contain a unique value for each record.

In [ ]:
df7 = df6.withColumn("id", monotonically_increasing_id()+1)
df7.select('id').show(5)

We can also perform SQL queries on spark dataframe using the `spark.sql()` function. But for that, we first need to register the dataframe as a table in the spark catalog. We can do this using the `createOrReplaceTempView()` spark dataframe method. It takes the name of the temporary table we'd like to register as argument. As this table is temporary, it can only be accessed from the specific SparkSession used to create the Spark dataframe.

In [ ]:
df7.createOrReplaceTempView('df_table')
spark.sql("select date, department, team, smv, over_time, incentive from df_table where department='sewing'").show(5)

#### Load Data

Once we have extracted and transformed our data, we might want to load it into the destination or store it somewhere. We will load it into the MongoDB database.

Data in MongoDB is represented and stored using JSON-style documents. In PyMongo we use dictionaries to represent documents.

In [ ]:
data = df7.toPandas()
documents = []
for i in range(len(data)):
        doc = data.iloc[i,:].to_dict()
        for keys in doc:
          if keys != 'date':
            if type(doc[keys]) not in [str]:
                doc[keys] = float(doc[keys])
        documents.append(doc)
documents[0:1]

If you would like to perform the data insertion step then please **create your own account** on MongoDB Atlas as given in the reference [here](https://cdn.iisc.talentsprint.com/CDS/DB_Connect_Docs/Assignment_MongoDB_Connect.pdf) and change the credentials and run the below code by uncommenting it.

In [ ]:
### new_document = coll.insert_many(documents)

Also, we can store the data in other formats like json, csv, and parquet and read it back whenever required.

**Store the dataframe as a `json file`**

In [ ]:
df7.write.format("json").mode("overwrite").save('transformed_json_data.json')

**Read data from `json` to spark dataframe**

In [ ]:
df_json = spark.read.format("json").load('transformed_json_data.json')

In [ ]:
df_json.show(5)

**Store the dataframe as a `csv file`**

In [ ]:
df7.write.format("csv").mode("overwrite").option("header", "true").save("transformed_csv_data.csv")

**Read data from `csv` to spark dataframe**

In [ ]:
df_csv = spark.read.format("csv").option("header", "true").load('transformed_csv_data.csv')

In [ ]:
df_csv.show(5)

**Use Parquet files to store data**

Parquet uses snappy compression to compress the data. If the DataFrame is written as Parquet, the schema is preserved as part of the Parquet metadata.

To know more about parquet file format click [here](https://cdn.iisc.talentsprint.com/CDS/Assignments/Module5/ellicium_com_blog_parquet_file_format_structure_teevel_2C_20the_stored_20in_20the_20footer_20section.pdf).

In [ ]:
df7.write.format("parquet").mode("overwrite").save("transformed_parquet_data")

**Read data from Parquet file**

We don't have to specify the schema here since it's stored as part of the Parquet metadata.

In [ ]:
df_parquet = spark.read.format("parquet").load("transformed_parquet_data")

In [ ]:
df_parquet.show(5)